<a href="https://colab.research.google.com/github/IAT-ComputationalCreativity-Spring2025/Week3-Rule-Based-Systems/blob/main/markov_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Markov Models Text Generation

This notebook introduces Markov chains for text generation. We'll build a simple
text generator that learns patterns from input text and generates new text with
similar statistical properties.

In [10]:
# First, let's import our required libraries

# Import defaultdict class from Python's built-in collections module
# defaultdict is a special kind of dictionary that provides a default value for a key that deosn not exist, preventing KeyError
from collections import defaultdict

# Import Python's built-in random module
# ramdom provides functions for generating random numbers, selecting random elements, and performing other randomization tasks
import random

## Building the Markov Chain

A Markov chain represents sequences of states where the probability of each state
depends only on the previous state(s). In our case, each state will be a sequence
of words, and we'll predict the next word based on this sequence.

In [2]:
# FUNCTION DEFINITION
# build_markov_chain: Function that creates a Markov chain from a given input text
# text: A string input from which the function builds the chain
# order: The number of words used as the state (context) to predict the next word
# The default value is 2
def build_markov_chain(text, order=2):

    # DOCSTRING
    """
    Build a Markov chain from input text.

    Args:
        text (str): Input text to learn from
        order (int): Number of words to use as state (context)

    Returns:
        dict: Mapping from state tuples to lists of possible next words
    """

    # INTIALIZING THE CHAIN
    # defaultdict(list): Intializes an empty dictionary where keys are tuples representing states and values are lists of possible next words
    # Using defaultdict(list) ensures that if a key is missing, it automatically creates an empty list instead of raising a KeyError
    chain = defaultdict(list)

    # SPLITTING THE TEXT
    # Splits the input text into a list of words using whitespace as the delimiter
    words = text.split()

    # BUILDING THE MARKOV CHAIN
    # Loop that iterates through the words, stopping at len(words) - order to ensure it doesn't go out of bounds when accessing words[i+order]
    for i in range(len(words) - order):

        # EXTRACTING THE STATE AND NEXT WORD
        # State: A tuple of order words (default if 2 words)
        # Create state tuple from current words
        state = tuple(words[i:i+order])

        # next_word: The word that follows the state
        next_word = words[i+order]

        # STORING THE RELATIONSHIP IN THE CHAIN
        # Adds the next_word to the list of possible next words fot this state
        chain[state].append(next_word)

    # RETURNING THE MARKOV CHAIN
    # Return the generated Markov chain, which is a dictionary mapping state tuples to lists of possible next words
    return chain

## Generating Text

Now we'll use our Markov chain to generate new text. We'll randomly select from
the possible next words at each step.

In [3]:
def generate_text(chain, num_words=30):
    """
    Generate new text using the Markov chain.

    Args:
        chain (dict): Markov chain mapping states to possible next words
        order (int): Length of state tuples
        num_words (int): Number of words to generate

    Returns:
        str: Generated text
    """
    # Start with a random state from the chain
    words = list(random.choice(list(chain.keys())))
    order = len(list(chain.keys())[0])

    for _ in range(num_words - order):
        state = tuple(words[-order:])
        if state in chain:
            next_word = random.choice(chain[state])
            words.append(next_word)
        else:
            break

    return ' '.join(words)

## Part 3: Basic Example

Let's try our text generator with a simple example.

In [4]:
# Sample text
sample_text = """
The quick brown fox jumps over the lazy dog.
A quick brown dog jumps over the lazy fox.
The lazy fox sleeps while the quick brown dog watches.
"""

# Build the chain
chain = build_markov_chain(sample_text)

# Examine the chain
for state, words in chain.items():
    print(f"{' '.join(state)} -> {words}")

The quick -> ['brown']
quick brown -> ['fox', 'dog', 'dog']
brown fox -> ['jumps']
fox jumps -> ['over']
jumps over -> ['the', 'the']
over the -> ['lazy', 'lazy']
the lazy -> ['dog.', 'fox.']
lazy dog. -> ['A']
dog. A -> ['quick']
A quick -> ['brown']
brown dog -> ['jumps', 'watches.']
dog jumps -> ['over']
lazy fox. -> ['The']
fox. The -> ['lazy']
The lazy -> ['fox']
lazy fox -> ['sleeps']
fox sleeps -> ['while']
sleeps while -> ['the']
while the -> ['quick']
the quick -> ['brown']


In [5]:
# Generate some text
print("Generated text:")
print(generate_text(chain))

Generated text:
jumps over the lazy fox. The lazy fox sleeps while the quick brown dog jumps over the lazy dog. A quick brown dog jumps over the lazy dog. A quick


## Student Tasks

1. Basic Implementation:
   - Try changing the order parameter in build_markov_chain
   - What happens with order=1 vs order=3?

In [6]:
# Task 1: Experiment with different orders
print("\nOrder 1:")
chain1 = build_markov_chain(sample_text, order=1)
print(generate_text(chain1))

print("\nOrder 3:")
chain3 = build_markov_chain(sample_text, order=3)
print(generate_text(chain3))


Order 1:
jumps over the quick brown dog watches.

Order 3:
fox. The lazy fox sleeps while the quick brown dog watches.


2. Use Your Own Text:
   Below, try using a different text source. You could use:
   - Song lyrics
   - Book excerpts
   - Movie quotes

In [7]:
# Task 2: Add your own text here
your_text = """
[Replace this with your own text]
Example:
To be, or not to be, that is the question:
Whether 'tis nobler in the mind to suffer
The slings and arrows of outrageous fortune...
"""

3. Advanced Implementation:
   Add temperature-based sampling to control randomness

In [8]:
def generate_text_with_temperature(chain, temperature=1.0, num_words=30):
    """
    Generate text with temperature-based sampling.
    Lower temperature = more conservative/predictable
    Higher temperature = more random/creative

    Args:
        chain (dict): Markov chain
        temperature (float): Controls randomness (0.1 to 2.0 recommended)
        order (int): Length of state tuples
        num_words (int): Number of words to generate
    """
    words = list(random.choice(list(chain.keys())))
    order = len(list(chain.keys())[0])

    for _ in range(num_words - order):
        state = tuple(words[-order:])
        if state in chain:
            # Count frequencies of next words
            next_words = chain[state]
            word_counts = defaultdict(int)
            for word in next_words:
                word_counts[word] += 1

            # Apply temperature
            weights = [count ** (1.0 / temperature) for count in word_counts.values()]
            total = sum(weights)
            weights = [w/total for w in weights]

            # Choose next word based on weighted probabilities
            next_word = random.choices(list(word_counts.keys()), weights=weights)[0]
            words.append(next_word)
        else:
            break

    return ' '.join(words)

# Try different temperatures
print("\nLow temperature (more predictable):")
print(generate_text_with_temperature(chain, temperature=0.3))

print("\nHigh temperature (more random):")
print(generate_text_with_temperature(chain, temperature=2.0))


Low temperature (more predictable):
dog jumps over the lazy fox. The lazy fox sleeps while the quick brown dog watches.

High temperature (more random):
lazy fox. The lazy fox sleeps while the quick brown dog jumps over the lazy fox. The lazy fox sleeps while the quick brown fox jumps over the lazy dog.


## Challenge Tasks:

1. Implement a function to analyze the Markov chain:
   - Count the number of unique states
   - Find the most common transitions
   - Calculate the average number of possible next words for each state

In [9]:
def analyze_chain(chain):
    """
    Analyze properties of the Markov chain.

    Args:
        chain (dict): Markov chain to analyze
    """
    num_states = len(chain)
    total_transitions = sum(len(next_words) for next_words in chain.values())
    avg_transitions = total_transitions / num_states if num_states > 0 else 0

    # Find most common next word for each state
    most_common = {}
    for state, next_words in chain.items():
        word_counts = defaultdict(int)
        for word in next_words:
            word_counts[word] += 1
        most_common[state] = max(word_counts.items(), key=lambda x: x[1])

    print(f"Number of unique states: {num_states}")
    print(f"Average transitions per state: {avg_transitions:.2f}")
    print("\nMost common transitions:")
    for state, (word, count) in list(most_common.items())[:5]:  # Show top 5
        print(f"{' '.join(state)} -> {word} (count: {count})")

# Analyze our chain
print("\nChain Analysis:")
analyze_chain(chain)


Chain Analysis:
Number of unique states: 20
Average transitions per state: 1.30

Most common transitions:
The quick -> brown (count: 1)
quick brown -> dog (count: 2)
brown fox -> jumps (count: 1)
fox jumps -> over (count: 1)
jumps over -> the (count: 2)


## Further Exploration:

Other ideas to try:
1. Modify the code to preserve punctuation
2. Add start-of-sentence and end-of-sentence tokens
3. Implement bi-directional generation
4. Create a chain that works with characters instead of words
5. Add input validation and error handling